In [29]:
!pip install -e ./task_model 
# !pip install tensorflow-addons

Obtaining file:///C:/Users/NPDan/Documents/GitHub/task_challenge/src/task_model
  Attempting uninstall: task-model
    Found existing installation: task-model 0.1.0
    Uninstalling task-model-0.1.0:
      Successfully uninstalled task-model-0.1.0
  Running setup.py develop for task-model


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanSquaredError, MeanAbsolutePercentageError, MeanSquaredLogarithmicError
from tensorflow.image import rgb_to_grayscale, image_gradients

from task_model.task_unet_model import MobileNetV2_UNet_MeanMask as unet_model

import json
from skimage import io, morphology
from pathlib import Path
from datetime import datetime

import shutil

In [3]:
# set some base variables up
DEFAULT_IMAGE_SIZE = [512, 512, 3]
DEFAULT_BATCH = 4
DEFAULT_NUM_CHANNELS = 3

# designate model name identifier, if any
DEFAULT_SAVE_MODEL = True
model_savebase = 'meanMask_model_MSE_'

# set some path variables up
base_dir = Path('C:/Users/NPDan/Documents/GitHub/task_challenge/data')

# test setup on cpu, Omit for train
tf.config.set_visible_devices([], 'GPU')

# set seed for RNG
tf.random.set_seed(4321)


In [4]:
# pull in the dataset dataframes
train_df_path = base_dir / 'train_df.json'
with open(train_df_path, 'r') as fp:
    train_df = pd.read_json(fp)
valid_df_path = base_dir / 'valid_df.json'
with open(valid_df_path, 'r') as fp:
    valid_df = pd.read_json(fp)
test_df_path = base_dir / 'test_df.json'
with open(test_df_path, 'r') as fp:
    test_df = pd.read_json(fp)

Alright, first things first, we need to construct a dataset. That means parsing through each image in the training, test and validation datasets and copying them to a new folder

In [5]:
mm_dataset_dir = base_dir / 'datasets_mm'
if not mm_dataset_dir.is_dir():
    mm_dataset_dir.mkdir()

# make and validate sub dirs
mm_train_dir, mm_valid_dir, mm_test_dir = mm_dataset_dir / 'train', mm_dataset_dir / 'valid', mm_dataset_dir / 'test'
for sub_dir in [mm_train_dir, mm_valid_dir, mm_test_dir]:
    if not sub_dir.is_dir():
        sub_dir.mkdir()
        
# define sub-sub dirs
img_sub_dir, msk_sub_dir = 'images', 'masks'

In [6]:
class_key = 'building'
drop_keys = ['building', 'boundary', 'background']
for data_df, dst_parent in zip([train_df, valid_df, test_df], [mm_train_dir, mm_valid_dir, mm_test_dir]):
    # define dirs
    src_msk_dir = Path(data_df[class_key].unique()[0])
    src_img_dir = Path(data_df[class_key].unique()[0].replace('masks','images'))
    dst_msk_dir = dst_parent / msk_sub_dir
    dst_img_dir = dst_parent / img_sub_dir
    
    # loop over, copy images
    # for ii, (src_dir, dst_dir) in enumerate(zip([src_img_dir, src_msk_dir], [dst_img_dir, dst_msk_dir])):
    #     shutil.copytree(src_dir, dst_dir)
    data_df[img_sub_dir] = str(dst_img_dir)
    data_df[msk_sub_dir] = str(dst_msk_dir)
    data_df.drop(drop_keys, axis='columns', inplace=True)

Great, initial images are copied, now let's calculate the mask_mean image by taking the mean for each channel 
from the mask area

First, let's write a quick method that does this calculation

In [7]:
def get_mean_mask(img, msk):
    # copy img, update with mean of mask area
    img_out = img.copy()
    for ch in range(img.shape[2]):
        img_out[msk,ch] = np.round(np.mean(img[msk,ch])).astype('uint8')
    return img_out

In [8]:
mm_sub_dir = 'mean_mask'
for data_df in [train_df, valid_df, test_df]:
    # set up new paths
    src_img_dir = Path(data_df[img_sub_dir].unique()[0])
    src_msk_dir = Path(data_df[msk_sub_dir].unique()[0])
    dst_img_dir = src_img_dir.parent / mm_sub_dir
    data_df[mm_sub_dir] = str(dst_img_dir)
    
    # you know it
    if not dst_img_dir.is_dir():
        dst_img_dir.mkdir()
        
    # loop through images
    continue # comment out if needed
    for src_img_path in sorted(src_img_dir.glob('*.png')):
        # paths yo
        src_msk_path = src_msk_dir / src_img_path.name
        dst_img_path = dst_img_dir / src_img_path.name
        
        # read in images, calculate mean_mask
        src_img = io.imread(src_img_path)
        src_msk = io.imread(src_msk_path) > 0 # bool conversion for indexing
        dst_img = get_mean_mask(src_img, src_msk)
        io.imsave(dst_img_path, dst_img)

Alright, we have our dataset, let's get our dataloaders set up and get crankin'

In [9]:
# let's reuse our janky image read methods to parse in our x, y paths
def read_data(path, float_convert=True, scale_fctr=255., return_3D=True):
    # type enforcement for consistency
    if not isinstance(path, (Path, str)):
        path = path.decode()
    if float_convert:
        data = _read_float(path, scale_fctr)
    else:
        data = _read_u8(path, scale_fctr)
    if return_3D:
        data = np.atleast_3d(data)
    return data

def _read_float(path, scale_fctr=1.):
    x = io.imread(path) / scale_fctr
    return x.astype('float32') 

def _read_u8(path, scale_fctr=1):
    x = io.imread(path) / scale_fctr
    return x.astype('uint8')

In [10]:
# let's get our datasets
def tf_parse(x, y, image_size=DEFAULT_IMAGE_SIZE[0]):
    def _parse(x, y):
        x = read_data(x, True)
        y = read_data(y, True)
        return x, y
    
    # NOTE: need to be careful, not 100% what these are doing
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([image_size, image_size, 3])
    y.set_shape([image_size, image_size, 3])
    return x, y

def tf_dataset(x, y, batch=DEFAULT_BATCH):
    # define dataset size, map parse function, batch and allow repeat loop    
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [11]:
# get our path vectors
train_x = sorted([str(p) for p in Path(train_df[img_sub_dir].unique()[0]).glob('*.png')])
train_y = sorted([str(p) for p in Path(train_df[mm_sub_dir].unique()[0]).glob('*.png')])
valid_x = sorted([str(p) for p in Path(valid_df[img_sub_dir].unique()[0]).glob('*.png')])
valid_y = sorted([str(p) for p in Path(valid_df[mm_sub_dir].unique()[0]).glob('*.png')])
test_x = sorted([str(p) for p in Path(test_df[img_sub_dir].unique()[0]).glob('*.png')])
test_y = sorted([str(p) for p in Path(test_df[mm_sub_dir].unique()[0]).glob('*.png')])

# get our tf datasets
train_dataset = tf_dataset(train_x, train_y, batch=DEFAULT_BATCH)
valid_dataset = tf_dataset(valid_x, valid_y, batch=DEFAULT_BATCH)
test_dataset = tf_dataset(test_x, test_y, batch=DEFAULT_BATCH)

In [12]:
# sanity check, is our data the right shape
train_dataset.element_spec, len(test_x) # Huzzah!


((TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None)),
 175)

In [13]:
x, y = next(train_dataset.as_numpy_iterator())
x.shape, y.shape

((4, 512, 512, 3), (4, 512, 512, 3))

In [24]:
DEFAULT_SMOOTH_EPS = 1e-7 # avoid divide by zero issues
def grad_weighted_mse(y_true, y_pred, smooth=DEFAULT_SMOOTH_EPS):
    gray_img = rgb_to_grayscale(y_true)
    gx, gy = image_gradients(gray_img)
    
    # copy over zeros w/ nn for gx, gy
    # gx[:,:,-1,:], gy[:,-1,:,:] = gx[:,:,-2,:], gy[:,-2,:,:]
    grad_img = gx + gy
    grad_img /= tf.reduce_max(grad_img, axis=(1,2,3))
    #for i in range(grad_image.shape[0]):
    #    grad_img[i,:,:,:] /= tf.reduce_max(grad_img[i,:,:,:])
    
    # apply weights to 
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    
    return tf.reduce_mean(tf.square(y_true - y_pred) * tf.keras.layers.Flatten()(grad_img))


Moment of truth... let's instantiate our optimizer and callbacks and start fittin'

In [25]:
DEFAULT_EPS = 1e-6
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False, min_delta = 0.001),
]

# SET training steps - validation data includes auto-augmentations
train_steps = len(train_x)//DEFAULT_BATCH
valid_steps = len(valid_x)//DEFAULT_BATCH
if len(train_x) % DEFAULT_BATCH != 0:
    train_steps += 1
if len(valid_x) % DEFAULT_BATCH != 0: 
    valid_steps += 1

In [26]:
model = unet_model(num_channels=DEFAULT_NUM_CHANNELS)
# model.summary()

In [30]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         encoder_input[0][0]              
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 256, 256, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [27]:
DEFAULT_LR = 0.1
DEFAULT_EPOCHS = 20
opt = tf.keras.optimizers.Adam(DEFAULT_LR)
metrics = [
    MeanAbsoluteError(),
    MeanAbsolutePercentageError(),
]
model.compile(loss=grad_weighted_mse ,
              optimizer=opt,
              metrics=metrics
             )

In [28]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=DEFAULT_EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks,
)

Epoch 1/20


InvalidArgumentError:  Incompatible shapes: [4,786432] vs. [4,1048576]
	 [[node gradient_tape/grad_weighted_mse/mul/BroadcastGradientArgs (defined at <ipython-input-28-796b60a4fbfa>:1) ]] [Op:__inference_train_function_20491]

Function call stack:
train_function


In [ ]:
def visualize_image_and_mean_mask(x_img, y_true, y_pred, cmap_name='gray', show_axis=False, title_prefix = ''):
    fig = plt.figure(figsize=(15,40))
    
    # plot original
    plt.subplot(1,4,1)
    plt.title(' '.join((title_prefix, 'Image')))
    plt.imshow(x_img, vmin=0, vmax=1)

    # plot mean_mask
    plt.subplot(1,4,2)
    plt.title(' '.join((title_prefix, 'MeanMask')))
    plt.imshow(y_true, vmin=0, vmax=1)

    # plot mean_mask
    plt.subplot(1,4,3)
    plt.title(' '.join((title_prefix, 'Predicted')))
    plt.imshow(y_pred, vmin=0, vmax=1)
    
    # plot difference
    plt.subplot(1,4,4)
    plt.title(' '.join((title_prefix, 'Difference')))
    plt.imshow(np.mean(y_true - y_pred, axis=2), vmin=-1, vmax=1, cmap=cmap_name)

    plt.show()

In [ ]:
# let's check the output
avg_mse, avg_mpe = [], []
for ii, (x, y) in enumerate(zip(test_x, test_y)):
    x = read_data(Path(x), True)
    y = read_data(Path(y), True, 255.)
    with tf.device('/cpu:0'):
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    mse = np.mean(np.square(y_pred.flatten() - y.flatten()))
    mpe = np.mean(np.abs(y_pred.flatten() - y.flatten()) / (x.flatten() + DEFAULT_EPS))
    if ii % 30 == 0:
        visualize_image_and_mean_mask(x, y, y_pred)
        print(f'MSE: {mse:.3f}  MPE: {mpe:.3f}')

    avg_mse.append(mse)
    avg_mpe.append(mpe)


print(f'Final MSE: {np.mean(avg_mse):.3f}')
print(f'Final MPE: {np.mean(avg_mpe):.3f}')


In [ ]:
x[200,200,:], y[200,200,:], y_pred[200,200,:]